### This file is just used for some data cleaning/exploration and debugging purposes

In [16]:
import pandas as pd
from streaming_optimizer import optimize_streaming_packages, preprocess_data, print_solver_results
from datetime import datetime

In [66]:
games_raw = pd.read_csv('data/games_cleaned.csv') # some games and when they are played
streaming_packages_raw = pd.read_csv('data/bc_streaming_package.csv') # streaming packages with price
streaming_offers_raw = pd.read_csv('data/bc_streaming_offer.csv')  # which service offers which game, live? on-demand-highlights?

package_offers = streaming_packages_raw.merge(
    streaming_offers_raw, left_on='id', right_on='streaming_package_id', how='right')

merged = games_raw.merge(package_offers, left_on='id', right_on='game_id', how='right'
                         , suffixes=('_game', '_offer'))
merged[merged['team_home'] == 'Bologna FC']
merged[merged['id_game'] == 8548]
pd.to_datetime(games_raw['starts_at']).describe() # 28.07.2023 - 01.06.2025
len(games_raw['tournament_name'].unique())
games_raw.head(3)

,id,team_home,team_away,starts_at,tournament_name
0,1,Deutschland,Schottland,2024-06-14 19:00:00,Europameisterschaft 2024
1,2,Ungarn,Schweiz,2024-06-15 13:00:00,Europameisterschaft 2024
2,3,Spanien,Kroatien,2024-06-15 16:00:00,Europameisterschaft 2024


In [ ]:
import json

team_home = games_raw['team_home'].unique()
team_away = games_raw['team_away'].unique()
clubs = set(team_home).union(set(team_away))
with open('teams.json', 'w', encoding='utf-8') as f:
    #json.dump(list(clubs), f, ensure_ascii=False, indent=4)

In [29]:
game_ids_of_interest = games_raw[(games_raw['team_home'] == 'Bologna FC')]['id'].unique() #list(range(31, 39)) # Input from the FE

In [72]:
merged[merged['team_home'] == 'FC Dinamo Batumi' | merged['team_away'] == 'FC Dinamo Batumi']

,id_game,team_home,team_away,starts_at,tournament_name,id_offer,name,monthly_price_cents,monthly_price_yearly_subscription_in_cents,game_id,streaming_package_id,live,highlights
29185,7782,FC Salzburg,FC Twente Enschede,2024-08-06 18:45:00,UEFA Champions League 24/25,37,DAZN - World,999.0,699,7782,37,0,1
29186,7782,FC Salzburg,FC Twente Enschede,2024-08-06 18:45:00,UEFA Champions League 24/25,38,DAZN - Super Sports,2499.0,1999,7782,38,0,1
29187,7782,FC Salzburg,FC Twente Enschede,2024-08-06 18:45:00,UEFA Champions League 24/25,39,DAZN - Free,0.0,0,7782,39,0,1


In [31]:
p = preprocess_data(game_ids_of_interest, streaming_offers_raw, streaming_packages_raw, games_raw)
results = optimize_streaming_packages(p['packages'], p['games'], p['game_dates'], p['C_month'], p['C_year'], p['P_g'])

print('No package in DB for these games:', p['games_with_no_offers'])

print_solver_results(results)

d:\Private Workspace\streaming-package-comperator\backend\streaming_optimizer.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_packages['yearly_price'] = filtered_packages['monthly_price_yearly_subscription_in_cents'] * 12


No package in DB for these games: [np.int64(8548)]
Solver Status: Optimal
Total Cost of Selected Subscriptions: 39984.0

Active Monthly Subscriptions:
  Package: 3, Start Date: 2024-08-31
  Package: 3, Start Date: 2024-09-18
  Package: 3, Start Date: 2024-10-06
  Package: 3, Start Date: 2024-10-26
  Package: 3, Start Date: 2024-11-02
  Package: 3, Start Date: 2024-11-05
  Package: 3, Start Date: 2024-11-27
  Package: 13, Start Date: 2024-08-31
  Package: 13, Start Date: 2024-09-18
  Package: 13, Start Date: 2024-10-06
  Package: 13, Start Date: 2024-10-26
  Package: 13, Start Date: 2024-11-02
  Package: 13, Start Date: 2024-11-05
  Package: 13, Start Date: 2024-11-27
  Package: 16, Start Date: 2024-08-31
  Package: 16, Start Date: 2024-09-18
  Package: 16, Start Date: 2024-10-06
  Package: 16, Start Date: 2024-10-26
  Package: 16, Start Date: 2024-11-02
  Package: 16, Start Date: 2024-11-05
  Package: 16, Start Date: 2024-11-27
  Package: 38, Start Date: 2023-08-11
  Package: 38, Start

In [ ]:
## Preprocess data (make everthing as small as possible)

### Filter packages to include only relevant ones
# Identify relevant package IDs from offers
relevant_package_ids = streaming_offers_raw[streaming_offers_raw['game_id'].isin(game_ids_of_interest)]['streaming_package_id'].unique()

# Filter packages to include only relevant ones
filtered_packages = streaming_packages_raw[streaming_packages_raw['id'].isin(relevant_package_ids)]

filtered_packages['yearly_price'] = filtered_packages['monthly_price_yearly_subscription_in_cents'] * 12
filtered_packages = filtered_packages.drop(columns=['monthly_price_yearly_subscription_in_cents'])


games = games_raw[games_raw['id'].isin(game_ids_of_interest)]

# Filter offers to include only the games of interest
filtered_offers = streaming_offers_raw[streaming_offers_raw['game_id'].isin(game_ids_of_interest)]


# Extract unique package IDs
packages = filtered_packages['id'].unique().tolist()

# Create game_dates dictionary
games.loc[:, 'starts_at'] = pd.to_datetime(games['starts_at'])
game_dates = games.set_index('id')['starts_at'].apply(lambda x: x.date()).to_dict()


# Create C_month and C_year dictionaries, dropping packages with NA for the respective type
C_month = filtered_packages.dropna(subset=['monthly_price_cents']) \
    .set_index('id')['monthly_price_cents'].to_dict()
C_year = filtered_packages.dropna(subset=['yearly_price']) \
    .set_index('id')['yearly_price'].to_dict()


# Create P_g dictionary
P_g = filtered_offers.groupby('game_id')['streaming_package_id'].apply(list).to_dict()


## Remove all games with no offer

games_with_no_offers = []

for game_id in game_ids_of_interest:
    if game_id not in P_g:
        games_with_no_offers.append(game_id)

game_ids_of_interest = [game_id for game_id in game_ids_of_interest if game_id in P_g]

#print("Games with no offers:", games_with_no_offers)

results = optimize_streaming_packages(packages, game_ids_of_interest, game_dates, C_month, C_year, P_g)

if results is not None:
    # Print results
    print("Status:", results["status"])
    print("Total Cost:", results["total_cost"])
    print("Active Monthly Subscriptions:")
    for sub in results["active_monthly_subscriptions"]:
        print(f"  Package: {sub['package']}, Start Date: {sub['start_date'].strftime('%Y-%m-%d')}")
    print("Active Yearly Subscriptions:")
    for sub in results["active_yearly_subscriptions"]:
        print(f"  Package: {sub['package']}, Start Date: {sub['start_date'].strftime('%Y-%m-%d')}")
else:
    print("No results returned from optimize_streaming_packages function.")

C:\Users\tmari\AppData\Local\Temp\ipykernel_27356\406274566.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_packages['yearly_price'] = filtered_packages['monthly_price_yearly_subscription_in_cents'] * 12


Status: Optimal
Total Cost: 39984.0
Active Monthly Subscriptions:
  Package: 3, Start Date: 2024-08-31
  Package: 3, Start Date: 2024-09-18
  Package: 3, Start Date: 2024-10-06
  Package: 3, Start Date: 2024-10-26
  Package: 3, Start Date: 2024-11-02
  Package: 3, Start Date: 2024-11-05
  Package: 3, Start Date: 2024-11-27
  Package: 13, Start Date: 2024-08-31
  Package: 13, Start Date: 2024-09-18
  Package: 13, Start Date: 2024-10-06
  Package: 13, Start Date: 2024-10-26
  Package: 13, Start Date: 2024-11-02
  Package: 13, Start Date: 2024-11-05
  Package: 13, Start Date: 2024-11-27
  Package: 16, Start Date: 2024-08-31
  Package: 16, Start Date: 2024-09-18
  Package: 16, Start Date: 2024-10-06
  Package: 16, Start Date: 2024-10-26
  Package: 16, Start Date: 2024-11-02
  Package: 16, Start Date: 2024-11-05
  Package: 16, Start Date: 2024-11-27
  Package: 38, Start Date: 2023-08-11
  Package: 38, Start Date: 2023-09-24
  Package: 38, Start Date: 2023-10-31
  Package: 38, Start Date: 20

In [68]:
from streaming_optimizer import optimize_streaming_packages
from datetime import datetime, timedelta

# Example Input Data
packages = ["P1", "P2"]
games = ["G1", "G2", "G3", "G4", "G5", 'G6', 'G7']
game_dates = {
    "G1": datetime(2023, 1, 15),
    "G2": datetime(2023, 2, 20),
    "G3": datetime(2023, 3, 25),
    "G4": datetime(2023, 4, 10),
    "G5": datetime(2023, 4, 20),
    "G6": datetime(2024, 1, 10),
    "G7": datetime(2024, 3, 15),
}
C_month = {'P1': 20}
C_year = {'P1': 180}
P_g = {'G1': ['P1'], 'G2': ['P1'], 'G3': ['P1', 'P2'], 'G4': ['P1', 'P2'], 'G5': ['P1', 'P2'],
    'G6': ['P1'], 'G7': ['P1', 'P2']}
results = optimize_streaming_packages(packages, games, game_dates, C_month, C_year, P_g)


if results is not None:
    # Print results
    print("Status:", results["status"])
    print("Total Cost:", results["total_cost"])
    print("Active Monthly Subscriptions:")
    for sub in results["active_monthly_subscriptions"]:
        print(f"  Package: {sub['package']}, Start Date: {sub['start_date'].strftime('%Y-%m-%d')}")
    print("Active Yearly Subscriptions:")
    for sub in results["active_yearly_subscriptions"]:
        print(f"  Package: {sub['package']}, Start Date: {sub['start_date'].strftime('%Y-%m-%d')}")
else:
    print("No results returned from optimize_streaming_packages function.")



Status: Optimal
Total Cost: 100.0
Active Monthly Subscriptions:
  Package: P1, Start Date: 2023-01-15
  Package: P1, Start Date: 2023-02-20
  Package: P1, Start Date: 2023-03-25
  Package: P1, Start Date: 2024-01-10
  Package: P1, Start Date: 2024-03-15
Active Yearly Subscriptions:
